In [3]:
import numpy as np
from os import path, listdir, makedirs
from itertools import product

from brainpipe.system import study
from rectification_smooth import rectification

In [6]:
from brainpipe.feat.utils._feat import _manageWindow
from brainpipe.feature import power

st = study('Olfacto')

################################################################################################
# Define power settings :
bsl, width, step = None, 358, 51 #358, 51 #[(768,1024),(1024,1280)],[(1280,1536)]
baseline = 'No baseline' #['-500_0'] #['-1500_-1000','-1000_-500'] #'700ms'
################################################################################################
reps = ['Encoding_By_Odor']

for rep in reps:
    path2save = path.join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
    code = 'E' if rep.startswith('Encoding') else 'R'
    ###############################################################################
    if not path.exists(path2save):
        makedirs(path2save)
    ###############################################################################  
    files = [k for k in st.search('bipo_all_noWM_physFT.npz', folder=('database/'+rep+'/'))]
    for fi in files:
        # Dict with all power features
        kwargs = {}
        f = [3,7] # Frequency vector
        loadname = path.join(st.path, 'database/'+rep+'/', fi)
        mat = np.load(loadname,allow_pickle=True)
        print('all files in archive', mat.files)
        x, sf = mat['x'], 512
        n_elec, n_pts, n_trials = x.shape
        print ('--> compute power on : ', rep +'/'+ fi,'shape', x.shape, 'baseline:', baseline)

        # Compute and save power objects :
        filt_rect = rectification(x,sf,f,width,step)
        kwargs['xpow'] = filt_rect
        win_all, time = _manageWindow(x.shape[1], width=width, step=step)
        kwargs['time'] = np.array(time) / sf
        kwargs['labels'], kwargs['channels']= mat['Mai_RL'], mat['channels']
        kwargs['xyz'] = mat['xyz']
        save_x = path2save+ fi.replace('bipo_all_noWM_physFT.npz', 'bipo_all_'+code+'_6freqs_hilbert=ok.npz')
        np.savez(save_x, **kwargs)
        del kwargs['xpow']
        del kwargs, x, sf, n_elec, n_trials

-> Olfacto loaded
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_1_bipo_all_noWM_physFT.npz shape (61, 3584, 1) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_2_bipo_all_noWM_physFT.npz shape (61, 3584, 3) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/LEFC_odor_14_bipo_all_noWM_physFT.npz shape (27, 3584, 6) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/LEFC_odor_15_bipo_all_noWM_physFT.npz shape (27, 3584, 11) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odo

all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_3_bipo_all_noWM_physFT.npz shape (61, 3584, 3) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_4_bipo_all_noWM_physFT.npz shape (61, 3584, 1) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/FERJ_odor_16_bipo_all_noWM_physFT.npz shape (32, 3584, 3) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/FERJ_odor_17_bipo_all_noWM_physFT.npz shape (32, 3584, 9) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/LEFC_odor_2_bipo_

#### Concatenate Power by odor in order to creat High and Low groups
New weighted episodic score (changs for FERJ and PIRJ)

In [7]:
import numpy as np
from brainpipe.system import study
from utils import odor_groups_3wgth as dict_
from os.path import join, exists
from os import makedirs
from itertools import product

st = study('Olfacto')
phases, RT_type = ['E'], 'RT_rec_first'
PATH = join(st.path, 'feature/TPSim_power_data/')
PATH_OD = join(PATH, 'Power_all_elecs_E_R_by_odor_all_elecsFT/')
od_name = join(PATH_OD, '{}_odor_{}_bipo_all_{}_6freqs_hilbert=ok.npz')
PATH_COND = join(PATH, 'Power_all_elecs_E_R_by_cond_3groups_v=1_elecs=all/')
save_name = join(PATH_COND, '{}_odor_{}_{}_6freqs_hilbert=ok.npz')

if not exists(PATH_COND):
    makedirs(PATH_COND)
    
for su,phase in product(dict_,phases):
    for cond in dict_[su]:
        print('>>> processing:', su, phase, cond)
        pow_data = np.array([])
        for od in dict_[su][cond]:
            mat = np.load(od_name.format(su,od,phase),allow_pickle=True)
            pow_data = np.concatenate((pow_data, mat['xpow']),axis=-1) if np.size(pow_data) else mat['xpow']
            print(su, od, cond,pow_data.shape)
        dict_pow = {}
        for file in mat.files:
            dict_pow[file] = mat[file]
        dict_pow['xpow'] = pow_data
        np.savez(save_name.format(su,cond,phase),**dict_pow)

-> Olfacto loaded
>>> processing: CHAF E low
CHAF 1 low (1, 61, 64, 1)
CHAF 2 low (1, 61, 64, 4)
CHAF 4 low (1, 61, 64, 5)
>>> processing: CHAF E mid
CHAF 5 mid (1, 61, 64, 1)
CHAF 8 mid (1, 61, 64, 4)
>>> processing: CHAF E high
CHAF 3 high (1, 61, 64, 3)
CHAF 7 high (1, 61, 64, 7)
CHAF 9 high (1, 61, 64, 12)
>>> processing: VACJ E low
VACJ 10 low (1, 39, 64, 3)
VACJ 11 low (1, 39, 64, 6)
VACJ 14 low (1, 39, 64, 9)
>>> processing: VACJ E mid
VACJ 12 mid (1, 39, 64, 2)
VACJ 17 mid (1, 39, 64, 4)
>>> processing: VACJ E high
VACJ 16 high (1, 39, 64, 3)
VACJ 13 high (1, 39, 64, 7)
VACJ 15 high (1, 39, 64, 9)
>>> processing: SEMC E low
SEMC 10 low (1, 53, 64, 7)
SEMC 7 low (1, 53, 64, 11)
SEMC 13 low (1, 53, 64, 21)
>>> processing: SEMC E mid
SEMC 11 mid (1, 53, 64, 4)
SEMC 12 mid (1, 53, 64, 10)
>>> processing: SEMC E high
SEMC 5 high (1, 53, 64, 3)
SEMC 9 high (1, 53, 64, 6)
SEMC 8 high (1, 53, 64, 9)
>>> processing: PIRJ E low
PIRJ 9 low (1, 18, 64, 6)
PIRJ 1 low (1, 18, 64, 13)
PIRJ 5 